<img src='./img/DataStore_EUMETSAT.png'/>

Copyright (c) 2022 EUMETSAT <br>
License: MIT

<hr>

<a href="./index.ipynb">← Index</a>
<br>
<a href="./2_Searching_and_filtering_products.ipynb">← Searching and filtering products</a><span style="float:right;">

# Downloading products

## What will this module teach you?

This module will show you how to:<br>
1. Download all products of specific **Collection ID and time**.
2. Download a product by **Product ID**.
3. Download a **single file** from a product.

## Selecting and filtering a collection (a recap)
As usual, we begin by importing our required modules.

In [21]:
import eumdac
import datetime
import shutil
import requests
import time

In [ ]:
curl -k -d "grant_type=cornkle" \
-H "Authorization: Basic Base64(iqt0toxQXCRd0wV8vJY5DMAOYLQa:lg7SYbPfz1p_KfKv7ql_fLb7NsQa)" \
https://api.eumetsat.int/token

Now, we have to authorize with our personal credentials to generate the token.

In [22]:
# Insert your personal key and secret into the single quotes

consumer_key = 'iqt0toxQXCRd0wV8vJY5DMAOYLQa'
consumer_secret = 'lg7SYbPfz1p_KfKv7ql_fLb7NsQa'

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

try:
    print(f"This token '{token}' expires {token.expiration}")
except requests.exceptions.HTTPError as error:
    print(f"Unexpected error: {error}")

This token '070ab075-72c5-3d82-89c7-bcb5247aad97' expires 2024-01-30 10:04:07.849146


<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
You can find your personal API credentials here: <a href="https://api.eumetsat.int/api-key/">https://api.eumetsat.int/api-key/</a>
</div>

Let's start to select a collection we want to browse through. For more information on determining which collections are available, see the previous tutorial, <a href="./1_Discovering_collections.ipynb">Discovering collections</a>

In [23]:
datastore = eumdac.DataStore(token)

try:    
    selected_collection = datastore.get_collection('EO:EUM:DAT:MSG:HRSEVIRI')
    print(f"{selected_collection} - {selected_collection.title}")
except eumdac.datastore.DataStoreError as error:
    print(f"Error related to the data store: '{error.msg}'")
except eumdac.collection.CollectionError as error:
    print(f"Error related to the collection: '{error.msg}'")
except requests.exceptions.ConnectionError as error:
    print(f"Error related to the connection: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

EO:EUM:DAT:MSG:HRSEVIRI - High Rate SEVIRI Level 1.5 Image Data - MSG - 0 degree


<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
EUMDAC (>=2.1.0) is retrying requests resulting into a HTTP 500 error automatically. If you are getting HTTP 40X errors you should review your requests, as they are often resulting from invalid user inputs. Find more information about EUMDAC errors, their causes and possible solutions, in our knowledge base: <a href="https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide#ID-Exception-handling">https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide#ID-Exception-handling</a>
</div>

The previous tutorial also showed us how to select a collection and filter it for products of interest using both time and space. As we are using a GEO product for our examples, there is no need for us to set a bounding box. However, we are going to perform a similar operation using time filtering so that we can set up a download. So, lets run our query to get our product IDs.

In [31]:
# Set sensing start and end time
start = datetime.datetime(2024, 1, 30, 8, 0)
end = datetime.datetime(2024, 1, 30, 18, 0)

# Retrieve datasets that match our filter
products = selected_collection.search(
    dtstart=start,
    dtend=end)

for product in products:
    try:
        print(product)
    except eumdac.collection.CollectionError as error:
        print(f"Error related to the collection: '{error.msg}'")
    except requests.exceptions.ConnectionError as error:
        print(f"Error related to the connection: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")

MSG3-SEVI-MSG15-0100-NA-20240130085743.024000000Z-NA
MSG3-SEVI-MSG15-0100-NA-20240130084243.016000000Z-NA
MSG3-SEVI-MSG15-0100-NA-20240130082743.008000000Z-NA
MSG3-SEVI-MSG15-0100-NA-20240130081243.000000000Z-NA


In [32]:
products

<class 'eumdac.collection.SearchResults'>(EO:EUM:DAT:MSG:HRSEVIRI, {'dtend': '2024-01-30T18:00:00', 'sort': None, 'geo': None, 'publication': None, 'dtstart': '2024-01-30T08:00:00', 'type': None, 'sat': None, 'title': None, 'set': None, 'bbox': None})

Hopefully we have found some datasets. If not, please adapt your search parameters until you have. In the following sections we will look at ways of downloading these.

## Downloading products
Products can be downloaded by providing either their product ID, or a combination of their collection ID and the time of acquisition. We can download entire products, or specific file components (e.g. metadata only). The cells below give three **independant** examples for all these methods.

### Download all products from search

We have now found our relevant products. Now we would like to download all of them without having to select every single product. For this we can simply loop over the products and save the products as files in our directory.

In [33]:
cnt = 1
for product in products:
    
    if cnt == len(products):
        
        try:
            with product.open() as fsrc, \
                    open(fsrc.name, mode='wb') as fdst:
                shutil.copyfileobj(fsrc, fdst)
                print(f'Download of product {product} finished.')
        except eumdac.product.ProductError as error:
            print(f"Error related to the product '{product}' while trying to download it: '{error.msg}'")
        except requests.exceptions.ConnectionError as error:
            print(f"Error related to the connection: '{error.msg}'")
        except requests.exceptions.RequestException as error:
            print(f"Unexpected error: {error}")

    cnt += 1
    
    
print('All downloads are finished.')

Download of product MSG3-SEVI-MSG15-0100-NA-20240130081243.000000000Z-NA finished.
All downloads are finished.


### Download a single product

Sometimes, however, we only need one specific product from the collection. For this we just search for the product in the collection.

In [8]:
selected_product = datastore.get_product(
    product_id='MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA',
    collection_id='EO:EUM:DAT:MSG:HRSEVIRI')

try:
    print(selected_product) 
except eumdac.product.ProductError as error:
    print(f"Error related to the product: '{error.msg}'")
except requests.exceptions.ConnectionError as error:
    print(f"Error related to the connection: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA


In [9]:
try:
    with selected_product.open() as fsrc, \
            open(fsrc.name, mode='wb') as fdst:
        shutil.copyfileobj(fsrc, fdst)
    print(f'Download of product {selected_product} finished.')
except eumdac.product.ProductError as error:
    print(f"Error related to the product '{selected_product}' while trying to download it: '{error.msg}'")
except requests.exceptions.ConnectionError as error:
    print(f"Error related to the connection: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

Download of product MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA finished.


### Download a single file from product

Products are often made up or many files. Sometimes we just want to have a single file. Let's see the files inside the product first.

In [10]:
for entry in selected_product.entries:
    try:
        print(entry)
    except eumdac.product.ProductError as error:
        print(f"Error related to the product: '{error.msg}'")
    except requests.exceptions.ConnectionError as error:
        print(f"Error related to the connection: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")

EOPMetadata.xml
MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA.nat
manifest.xml


Files within the product may be downloaded by providing the file's name, as demonstrated in the cell below.

In [11]:
try:
    with selected_product.open(entry='EOPMetadata.xml') as fsrc, \
            open(fsrc.name, mode='wb') as fdst:
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of file {fsrc.name} finished.')
except eumdac.product.ProductError as error:
    print(f"Error related to the product '{selected_product}' while trying to download it: '{error.msg}'")
except requests.exceptions.ConnectionError as error:
    print(f"Error related to the connection: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

Download of file EOPMetadata.xml finished.


And now we can open this file:

In [12]:
with open('EOPMetadata.xml') as f:
    print(f.read())

<?xml version="1.0" encoding="UTF-8"?>
<eum:EarthObservation xmlns:gml="http://www.opengis.net/gml/3.2"
                      xmlns:eop="http://www.opengis.net/eop/2.1"
                      xmlns:om="http://www.opengis.net/om/2.0"
                      xmlns:ows="http://www.opengis.net/ows/2.0"
                      xmlns:xlink="http://www.w3.org/1999/xlink"
                      xmlns:eu="http://www.eumetsat.int/sentinel"
                      xmlns:eum="http://www.eumetsat.int/eop/1.0"
                      gml:id="idSIPMetadata_2_1">
   <om:phenomenonTime>
      <gml:TimePeriod gml:id="phenomenonTime">
         <gml:beginPosition>2021-11-10T08:00:10.065000000Z</gml:beginPosition>
         <gml:endPosition>2021-11-10T08:12:42.766000000Z</gml:endPosition>
      </gml:TimePeriod>
   </om:phenomenonTime>
   <om:resultTime>
      <gml:TimeInstant gml:id="resultTime">
         <gml:timePosition>2021-11-10T08:12:42.766000000Z</gml:timePosition>
      </gml:TimeInstant>
   </om:resultTime>

Now we know how to download products!

<a href="./index.ipynb">← Index</a>
<br>
<a href="./2_Searching_and_filtering_products.ipynb">← Searching and filtering products</a><span style="float:right;">

<hr>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE.txt">MIT License</a> | <span style="float:right;"><a href="https://gitlab.eumetsat.int/eumetlab/data-services/">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact</a></span></p>